In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
import sys
import os

# Definição do caminho do arquivo
CAMINHO_ARQUIVO = 'titanic/train.csv'

# 1. Carregar o Dataset
try:
    df = pd.read_csv(CAMINHO_ARQUIVO)
    print(f"Dataset carregado com sucesso de: {CAMINHO_ARQUIVO}")
except FileNotFoundError:
    print(f"ERRO: O arquivo não foi encontrado no caminho: {CAMINHO_ARQUIVO}")
    print("Por favor, verifique o diretório 'titanic/' e o nome do arquivo.")
    df = None # Interrompe a execução lógica se falhar

if df is not None:
    print(f"Tamanho inicial do dataset: {len(df)}")

Dataset carregado com sucesso de: titanic/train.csv
Tamanho inicial do dataset: 891


In [21]:
# Célula 5: Aplicando as Transformações
if df is not None:
    # Copiar o dataframe para manter o original intacto
    df_modelo = df.copy()

    # ----------------------------------------
    # 1. ENGENHARIA DE FEATURES (Seu código, com pequenas adições)
    # ----------------------------------------
    print("--- Iniciando Engenharia de Features ---")

    # A. Extração do Título (Ótima estratégia!)
    df_modelo['Title'] = df_modelo['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    df_modelo['Title'] = df_modelo['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 
                                                     'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df_modelo['Title'] = df_modelo['Title'].replace({'Mlle':'Miss', 'Ms':'Miss', 'Mme':'Mrs'})
    
    # B. Criação do Tamanho da Família e Indicador 'IsAlone'
    df_modelo['FamilySize'] = df_modelo['SibSp'] + df_modelo['Parch'] + 1
    df_modelo['IsAlone'] = (df_modelo['FamilySize'] == 1).astype(str)

    # C. (NOVO) Discretização da Idade em faixas (Binning)
    # Árvores de decisão podem se beneficiar de faixas etárias bem definidas.
    df_modelo['AgeGroup'] = pd.cut(df_modelo['Age'], bins=[0, 12, 18, 60, 100], labels=['Criança', 'Adolescente', 'Adulto', 'Idoso'])

    print("Novas features ('Title', 'FamilySize', 'IsAlone', 'AgeGroup') criadas.")

    # ----------------------------------------
    # 2. TRATAMENTO DE VALORES NULOS
    # ----------------------------------------
    print("\n--- Iniciando Tratamento de Nulos ---")
    
    # A. Imputação de 'Age' baseada na mediana por Título (Muito bom!)
    mediana_idade_por_titulo = df_modelo.groupby('Title')['Age'].transform('median')
    df_modelo['Age'].fillna(mediana_idade_por_titulo, inplace=True)
    
    # B. Imputação de 'Embarked' com a moda
    moda_embarked = df_modelo['Embarked'].mode()[0]
    df_modelo['Embarked'].fillna(moda_embarked, inplace=True)

    # C. (NOVO) Preencher nulos em 'AgeGroup' que possam ter surgido
    # Se uma idade era nula, o AgeGroup também será. Preenchemos com 'Adulto' (a faixa mais comum).
    df_modelo['AgeGroup'].fillna('Adulto', inplace=True)

    print("Valores nulos de 'Age' e 'Embarked' tratados.")
    
    # ----------------------------------------
    # 3. SELEÇÃO E LIMPEZA FINAL
    # ----------------------------------------
    print("\n--- Realizando Limpeza Final ---")

    # Remover colunas originais que não serão usadas pelo modelo
    df_modelo.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], inplace=True)

    # ----------------------------------------
    # 4. CONVERSÃO DE TIPOS (Como você fez)
    # ----------------------------------------
    # Garantir que as features que devem ser tratadas como categorias sejam do tipo 'object'
    # É uma boa prática para seus algoritmos customizados saberem como tratar cada coluna.
    df_modelo['Pclass'] = df_modelo['Pclass'].astype('object')
    
    print("\n--- Dataset Final Pré-processado ---")
    print(df_modelo.head())
    print("\nVerificação de Nulos Finais:")
    print(df_modelo.isnull().sum())
    print("\nTipos de Dados Finais:")
    print(df_modelo.dtypes)

--- Iniciando Engenharia de Features ---
Novas features ('Title', 'FamilySize', 'IsAlone', 'AgeGroup') criadas.

--- Iniciando Tratamento de Nulos ---
Valores nulos de 'Age' e 'Embarked' tratados.

--- Realizando Limpeza Final ---

--- Dataset Final Pré-processado ---
   Survived Pclass     Sex   Age     Fare Embarked Title  FamilySize IsAlone  \
0         0      3    male  22.0   7.2500        S    Mr           2   False   
1         1      1  female  38.0  71.2833        C   Mrs           2   False   
2         1      3  female  26.0   7.9250        S  Miss           1    True   
3         1      1  female  35.0  53.1000        S   Mrs           2   False   
4         0      3    male  35.0   8.0500        S    Mr           1    True   

  AgeGroup  
0   Adulto  
1   Adulto  
2   Adulto  
3   Adulto  
4   Adulto  

Verificação de Nulos Finais:
Survived      0
Pclass        0
Sex           0
Age           0
Fare          0
Embarked      0
Title         0
FamilySize    0
IsAlone       

<>:12: SyntaxWarning: invalid escape sequence '\.'
<>:12: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_10870/4132545407.py:12: SyntaxWarning: invalid escape sequence '\.'
  df_modelo['Title'] = df_modelo['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
/tmp/ipykernel_10870/4132545407.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_modelo['Age'].fillna(mediana_idade_por_titulo, inplace=True)
/tmp/ipykernel_10870/4132545407.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Ser

In [22]:
# Célula 1: Preparação e Divisão dos Dados
# (Esta célula deve ser executada após a célula 5 do notebook anterior, que cria o df_modelo)

# Selecionando o DataFrame correto para o ID3
df_para_id3 = df_modelo.copy()

# Separando features (X) e alvo (y)
X = df_para_id3.drop('Survived', axis=1)
y = df_para_id3['Survived']

# Dividindo em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print("--- Dados Prontos ---")
print(f"Formato de X_train: {X_train.shape}")
print(f"Formato de X_test: {X_test.shape}")

--- Dados Prontos ---
Formato de X_train: (668, 9)
Formato de X_test: (223, 9)


In [23]:
from ID3 import ArvoreID3

# --- ETAPA 1: DEFINIR AS VARIÁVEIS PARA O TREINO ---
# Criar o dataframe de treino, que o método .treinar() espera
df_treino = pd.concat([X_train, y_train], axis=1)

# Definir a lista de atributos (colunas de features)
atributos = list(X_train.columns)
# Definir o nome da coluna alvo
coluna_alvo = 'Survived'


# --- ETAPA 2: TREINAR O MODELO ---
# Instanciar a sua classe
modelo_id3 = ArvoreID3()

# Chamar o método de treino
modelo_id3.treinar(df_treino, atributos, coluna_alvo)
print("✅ Modelo ID3 treinado com sucesso!")


# --- ETAPA 3: VISUALIZAR A ÁRVORE ---
print("\n" + "="*60)
print("ESTRUTURA DA ÁRVORE DE DECISÃO GERADA")
print("="*60)
modelo_id3.imprimir_arvore()
print("="*60 + "\n")


# --- ETAPA 4: FAZER PREVISÕES E AVALIAR ---
# Usar o novo método .predict() diretamente no conjunto de teste
print("📊 Realizando previsões no conjunto de teste...")
y_pred = modelo_id3.predict(X_test)
print("✅ Previsões concluídas!")

# Importar as funções de métric

# Calcular as métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# Imprimir os resultados
print("\n" + "="*50)
print("MÉTRICAS DE DESEMPENHO DO MODELO ID3")
print("="*50)
print(f"Acurácia (Accuracy): {accuracy:.4f}")
print(f"Precisão (Precision): {precision:.4f}")
print(f"Revocação (Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("="*50)

print("\nRelatório de Classificação Completo:")
print(classification_report(y_test, y_pred, zero_division=0))

✅ Modelo ID3 treinado com sucesso!

ESTRUTURA DA ÁRVORE DE DECISÃO GERADA
NÓ: Divisão por 'Fare'
  - Se 'Fare' == '90.0':
    --> PREDIÇÃO: Sobreviveu (classe=1)
  - Se 'Fare' == '10.5':
    NÓ: Divisão por 'Age'
      - Se 'Age' == '19.0':
        NÓ: Divisão por 'Pclass'
          - Se 'Pclass' == '2':
            NÓ: Divisão por 'Sex'
              - Se 'Sex' == 'male':
                NÓ: Divisão por 'Embarked'
                  - Se 'Embarked' == 'S':
                    NÓ: Divisão por 'Title'
                      - Se 'Title' == 'Mr':
                        NÓ: Divisão por 'FamilySize'
                          - Se 'FamilySize' == '1':
                            NÓ: Divisão por 'IsAlone'
                              - Se 'IsAlone' == 'True':
                                NÓ: Divisão por 'AgeGroup'
                                  - Se 'AgeGroup' == 'Adulto':
                                    --> PREDIÇÃO: Não Sobreviveu (classe=0)
      - Se 'Age' == '21.0':
        --

In [24]:
from C45 import ArvoreC45

# --- ETAPA 1: DEFINIR AS VARIÁVEIS PARA O TREINO ---
# Criar o dataframe de treino, que o método .treinar() espera
df_treino = pd.concat([X_train, y_train], axis=1)

# Definir a lista de atributos (colunas de features)
atributos = list(X_train.columns)
# Definir o nome da coluna alvo
coluna_alvo = 'Survived'


# --- ETAPA 2: TREINAR O MODELO ---
# Instanciar a sua classe
modelo_c45 = ArvoreC45()

# Chamar o método de treino
modelo_c45.treinar(df_treino, atributos, coluna_alvo)
print("✅ Modelo c45 treinado com sucesso!")


# --- ETAPA 3: VISUALIZAR A ÁRVORE ---
print("\n" + "="*60)
print("ESTRUTURA DA ÁRVORE DE DECISÃO GERADA")
print("="*60)
modelo_c45.imprimir_arvore()
print("="*60 + "\n")


# --- ETAPA 4: FAZER PREVISÕES E AVALIAR ---
# Usar o novo método .predict() diretamente no conjunto de teste
print("📊 Realizando previsões no conjunto de teste...")
y_pred = modelo_c45.predict(X_test)
print("✅ Previsões concluídas!")

# Importar as funções de métric

# Calcular as métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# Imprimir os resultados
print("\n" + "="*50)
print("MÉTRICAS DE DESEMPENHO DO MODELO c45")
print("="*50)
print(f"Acurácia (Accuracy): {accuracy:.4f}")
print(f"Precisão (Precision): {precision:.4f}")
print(f"Revocação (Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("="*50)

print("\nRelatório de Classificação Completo:")
print(classification_report(y_test, y_pred, zero_division=0))

✅ Modelo c45 treinado com sucesso!

ESTRUTURA DA ÁRVORE DE DECISÃO GERADA
NÓ: Divisão Categórica por 'Sex'
  - Se 'Sex' == 'female':
    NÓ: Divisão Numérica por 'FamilySize'
      - Se valor <= 7.000:
        NÓ: Divisão Numérica por 'Fare'
          - Se valor <= 6.987:
            --> PREDIÇÃO: Não Sobreviveu (classe=0)
          - Se valor > 6.987:
            NÓ: Divisão Categórica por 'Pclass'
              - Se 'Pclass' == '1':
                NÓ: Divisão Numérica por 'Age'
                  - Se valor <= 49.500:
                    NÓ: Divisão Categórica por 'Title'
                      - Se 'Title' == 'Mrs':
                        NÓ: Divisão Categórica por 'Embarked'
                          - Se 'Embarked' == 'S':
                            NÓ: Divisão Categórica por 'IsAlone'
                              - Se 'IsAlone' == 'False':
                                NÓ: Divisão Categórica por 'AgeGroup'
                                  - Se 'AgeGroup' == 'Adulto':
       

In [25]:
# Célula de Engenharia de Features (One-Hot Encoding)

print("--- Preparando dados com One-Hot Encoding para o CART ---")

# Usamos o df_modelo que já tem as features tratadas ('Title', 'AgeGroup', etc.)
# A função get_dummies converte todas as colunas de tipo 'object' em colunas binárias
df_ohe = pd.get_dummies(df_modelo, drop_first=True)

# Separando X e y novamente com os novos dados
X_ohe = df_ohe.drop('Survived', axis=1)
y_ohe = df_ohe['Survived']

# É crucial fazer um novo split para garantir a consistência
X_train_ohe, X_test_ohe, y_train_ohe, y_test_ohe = train_test_split(
    X_ohe, y_ohe, test_size=0.25, random_state=42, stratify=y_ohe
)

print(f"Formato original das features: {X_train.shape}")
print(f"Novo formato com One-Hot Encoding: {X_train_ohe.shape}")
print("\nNovas colunas criadas:", list(X_train_ohe.columns))

--- Preparando dados com One-Hot Encoding para o CART ---
Formato original das features: (668, 9)
Novo formato com One-Hot Encoding: (668, 16)

Novas colunas criadas: ['Age', 'Fare', 'FamilySize', 'Pclass_2', 'Pclass_3', 'Sex_male', 'Embarked_Q', 'Embarked_S', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare', 'IsAlone_True', 'AgeGroup_Adolescente', 'AgeGroup_Adulto', 'AgeGroup_Idoso']


In [27]:
from CART import ArvoreCART

# --- ETAPA 1: INSTANCIAR O MODELO ---
# Instanciamos o modelo com hiperparâmetros para controlar o crescimento da árvore
modelo_cart = ArvoreCART(max_depth=7, min_samples_leaf=5)


# --- ETAPA 2: TREINAR O MODELO ---
# O novo método .treinar() espera X e y separados, como no Scikit-learn.
# Não precisamos mais criar o 'df_treino' ou passar os nomes das colunas.
modelo_cart.treinar(X_train_ohe, y_train_ohe)
print("✅ Modelo CART treinado com sucesso!")


# --- ETAPA 3: VISUALIZAR A ÁRVORE ---
# Esta chamada agora funcionará se você adicionou o método ao seu arquivo .py
print("\n" + "="*60)
print("ESTRUTURA DA ÁRVORE DE DECISÃO GERADA PELO CART")
print("="*60)
modelo_cart.imprimir_arvore()
print("="*60 + "\n")


# --- ETAPA 4: FAZER PREVISÕES E AVALIAR ---
# Usamos o método .prever() que criamos
print("📊 Realizando previsões no conjunto de teste...")
y_pred = modelo_cart.prever(X_test_ohe)
print("✅ Previsões concluídas!")

# Calcular as métricas (com o erro de digitação corrigido)
accuracy = accuracy_score(y_test_ohe, y_pred)
precision = precision_score(y_test_ohe, y_pred, zero_division=0)
recall = recall_score(y_test_ohe, y_pred, zero_division=0) # Corrigido de 'zeo_division'
f1 = f1_score(y_test_ohe, y_pred, zero_division=0)

# Imprimir os resultados de forma clara
print("\n" + "="*50)
print("MÉTRICAS DE DESEMPENHO DO MODELO CART")
print("="*50)
print(f"Acurácia (Accuracy): {accuracy:.4f}")
print(f"Precisão (Precision): {precision:.4f}")
print(f"Revocação (Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("="*50)

# O classification_report é uma ótima forma de ver tudo de uma vez
print("\nRelatório de Classificação Completo:")
print(classification_report(y_test_ohe, y_pred, zero_division=0))

✅ Modelo CART treinado com sucesso!

ESTRUTURA DA ÁRVORE DE DECISÃO GERADA PELO CART
Se 'Title_Mr' <= 0.500:
  |-- V: Se 'Pclass_3' <= 0.500:
  |-- V:   |-- V: Se 'Title_Rare' <= 0.500:
  |-- V:   |-- V:   |-- V: Se 'Fare' <= 28.856:
  |-- V:   |-- V:   |-- V:   |-- V: Se 'Age' <= 49.000:
  |-- V:   |-- V:   |-- V:   |-- V:   |-- V: Se 'Age' <= 25.000:
  |-- V:   |-- V:   |-- V:   |-- V:   |-- V:   |-- V: --> PREDIÇÃO: Sobreviveu (classe=1)
  |-- V:   |-- V:   |-- V:   |-- V:   |-- V: Senão:
  |-- V:   |-- V:   |-- V:   |-- V:   |-- V:   |-- F: Se 'Age' <= 28.500:
  |-- V:   |-- V:   |-- V:   |-- V:   |-- V:   |-- F:   |-- V: --> PREDIÇÃO: Sobreviveu (classe=1)
  |-- V:   |-- V:   |-- V:   |-- V:   |-- V:   |-- F: Senão:
  |-- V:   |-- V:   |-- V:   |-- V:   |-- V:   |-- F:   |-- F: --> PREDIÇÃO: Sobreviveu (classe=1)
  |-- V:   |-- V:   |-- V:   |-- V: Senão:
  |-- V:   |-- V:   |-- V:   |-- V:   |-- F: --> PREDIÇÃO: Sobreviveu (classe=1)
  |-- V:   |-- V:   |-- V: Senão:
  |-- V:   |